# PHỤ LỤC A: KỸ THUẬT NUMPY NÂNG CAO

Trong phần phụ lục này, tôi sẽ đi sâu hơn vào thư viện NumPy dành cho việc tính toán mảng. Nội dung sẽ bao gồm các chi tiết nội bộ về kiểu dữ liệu ndarray và các thao tác cũng như thuật toán nâng cao với mảng.

Phụ lục này chứa các chủ đề đa dạng và không nhất thiết phải đọc theo thứ tự. Xuyên suốt các chương, tôi sẽ tạo dữ liệu ngẫu nhiên cho nhiều ví dụ, sử dụng bộ tạo số ngẫu nhiên mặc định trong mô-đun numpy.random.

In [1]:
# Importing các thư viện Python cần thiết
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [2]:
rng = np.random.default_rng(seed=12345)

## A.1 Cấu trúc bên trong của đối tượng ndarray


### Các thành phần của ndarray


Thư viện NumPy cung cấp kiểu dữ liệu ndarray cho phép diễn giải một khối dữ liệu có kiểu đồng nhất (liên tục hoặc có bước nhảy) dưới dạng một đối tượng mảng nhiều chiều. Kiểu dữ liệu, hay dtype, quyết định cách dữ liệu được hiểu là số thực, số nguyên, giá trị Boolean, hoặc các kiểu khác mà chúng ta đã tìm hiểu.

Một phần khiến ndarray trở nên linh hoạt là vì mỗi đối tượng mảng là một "view" có bước nhảy (strided view) trên một khối dữ liệu. Bạn có thể thắc mắc, ví dụ, tại sao cách truy cập mảng arr[::2, ::-1] lại không sao chép dữ liệu. Lý do là vì ndarray không chỉ là một khối bộ nhớ và kiểu dữ liệu; nó còn chứa thông tin về bước nhảy, cho phép mảng di chuyển qua bộ nhớ với các kích thước bước khác nhau.

Cụ thể hơn, bên trong ndarray bao gồm các thành phần sau:

Một con trỏ tới dữ liệu — tức là một khối dữ liệu nằm trong RAM hoặc trong một tệp ánh xạ bộ nhớ

Kiểu dữ liệu (dtype) mô tả các ô giá trị có kích thước cố định trong mảng

Một tuple biểu thị hình dạng (shape) của mảng

Một tuple các bước nhảy (strides) — các số nguyên biểu thị số byte cần “bước” để di chuyển một phần tử theo từng chiều

In [3]:
np.ones((10, 5)).shape

(10, 5)

In [4]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

### Hệ thống phân cấp kiểu dữ liệu (dtype)


In [5]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)

True

In [6]:
np.issubdtype(floats.dtype, np.floating)

True

In [7]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [8]:
np.issubdtype(ints.dtype, np.number)

True

## A.7 Viết hàm NumPy nhanh với Numba

### Giới thiệu Numba và @jit


In [9]:
import numpy as np

def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [10]:
x = rng.standard_normal(10_000_000)
y = rng.standard_normal(10_000_000)
%timeit mean_distance(x, y)

2.2 s ± 393 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit (x - y).mean()

43.5 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
import numba as nb

numba_mean_distance = nb.jit(mean_distance)

In [13]:
@nb.jit
def numba_mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [14]:
%timeit numba_mean_distance(x, y)

7.25 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
from numba import float64, njit
@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

### Tạo ufunc tùy chỉnh với @vectorize


In [16]:
from numba import vectorize
@vectorize
def nb_add(x, y):
    return x + y

In [17]:
x = np.arange(10)
nb_add(x, x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [18]:
nb_add.accumulate(x, 0)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

## A.9 Mẹo về Hiệu suất
 

### Tầm quan trọng của Bộ nhớ Liền kề (Contiguous Memory)


In [19]:
arr_c = np.ones((100, 10000), order='C')
arr_f = np.ones((100, 10000), order='F')
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [20]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [21]:
arr_f.flags.f_contiguous

True

In [22]:
%timeit arr_c.sum(1)

398 μs ± 53.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
%timeit arr_f.sum(1)

256 μs ± 53.1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [25]:
arr_c[:50].flags.contiguous

True

In [26]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False